# Evaluating Fairness in Machine Learning: Comparative Analysis and Benchmarking of Fairlearn and AIF360 | Adult Census Income

In [1]:
import os
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from src.constants import RANDOM_STATE, DEFAULT_MODEL_CONFIG

## Ensure reproducibility

Set random seeds for reproducibility.

In [2]:
np.random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
random.seed(RANDOM_STATE)

## Load data

In [5]:
TARGET = "income"
PROTECTED_ATTRIBUTE = "gender"

In [6]:
data = pd.read_csv("../data/adult.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [20]:
data

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [21]:
data.income = data.income.replace({">50K": 1, "<=50K": 0})

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_1636\3664918217.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.income = data.income.replace({">50K": 1, "<=50K": 0})


In [22]:
y = data.loc[:, TARGET]
z = data.loc[:, PROTECTED_ATTRIBUTE]
X = data.drop(columns=TARGET)

In [23]:
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [24]:
MODEL_CONFIG = dict(DEFAULT_MODEL_CONFIG, cat_features=X.select_dtypes("object").columns.to_list())

In [25]:
model = CatBoostClassifier(**MODEL_CONFIG)

In [26]:
model.fit(X_train, y_train)

0:	learn: 0.6834529	total: 48.4ms	remaining: 2m 25s
250:	learn: 0.3244321	total: 32.5s	remaining: 5m 55s
500:	learn: 0.3053939	total: 48.3s	remaining: 4m
750:	learn: 0.2975869	total: 1m 3s	remaining: 3m 10s
1000:	learn: 0.2922046	total: 1m 19s	remaining: 2m 38s
1250:	learn: 0.2868547	total: 1m 35s	remaining: 2m 13s
1500:	learn: 0.2835678	total: 1m 51s	remaining: 1m 50s
1750:	learn: 0.2816289	total: 2m 7s	remaining: 1m 31s
2000:	learn: 0.2798722	total: 2m 23s	remaining: 1m 11s
2250:	learn: 0.2783172	total: 2m 43s	remaining: 54.4s
2500:	learn: 0.2771612	total: 2m 59s	remaining: 35.9s
2750:	learn: 0.2760659	total: 3m 15s	remaining: 17.7s
2999:	learn: 0.2751247	total: 3m 31s	remaining: 0us


In [27]:
y_pred = model.predict(X_test)

## Detection

In [28]:
from fairlearn.metrics import demographic_parity_ratio, demographic_parity_difference, equalized_odds_ratio

In [29]:
demographic_parity_ratio(y_test, y_pred, sensitive_features=z_test)


0.30326717147521204

## Mitigation